In [7]:
# imports
import importlib as imp
import geopandas as gpd
import pandas as pd
import functions as f
# debug
imp.reload(f)
import streamlit as st

In [8]:
# check which forest area is along the way
forest_areas_with_bikelanes_wgs84 = gpd.read_parquet('dataset/processed/forest_areas_crossed_by_bikelane_wgs84.parquet')
forest_areas_with_bikelanes_dk = gpd.read_parquet('dataset/processed/forest_areas_crossed_by_bikelane_DK.parquet')
# Step 1: Ensure both GeoDataFrames are in the same CRS

### Get a route

In [32]:
import openrouteservice

# Replace 'your_api_key_here' with your actual OpenRouteService API key
client = openrouteservice.Client(key=st.secrets["OPENROUTSERVICE_API"])

# Define your coordinates
coordinates1 = [[11.581726, 55.606109], [11.699829, 55.592143]]  # Sjeland
coordinates4 = [[10.217285, 57.421294], [10.568847, 55.235288]] # super funky long, it will fail (no path)
coordinates5 = [[10.217285, 57.421294], [9.074707, 54.933455]]
coordinates6 = [[10.217285, 57.421294], [10.568847, 55.235288], [9.074707, 54.933455]]
coordinates2 = [[12.473907, 55.717571], [12.561798, 55.695132]] # in Copenhagen
coordinates7 = [[9.582405, 56.394366], [9.497604, 56.356956]]

coordinates = coordinates5

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=coordinates,
                           profile='cycling-regular',
                           format='geojson')

# The response 'routes' will be a GeoJSON object containing the route
gdf_routes_wgs84 = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes_wgs84.crs = 'EPSG:4326'
gdf_routes_wgs84.explore()

### We have to know which areas are checked already

In [49]:
# gdf_denmark = gdf_routes.to_crs(f.DENMARK_CRS)

##### TODO FROM HERE

In [33]:
imp.reload(f)
forest_areas_already_in_the_road_wg84 = f.get_only_areas_which_are_crossed_by_bikelane(forest_areas_with_bikelanes_wgs84, gdf_routes_wgs84)
forest_areas_already_in_the_road_dk = forest_areas_already_in_the_road_wg84.to_crs(f.DENMARK_CRS)

In [34]:
forest_areas_already_in_the_road_wg84.explore()

In [41]:
# reproject shortest path

BUFFER = 1000
ADD_MORE_AREA_TO_BE_VISITED = 10 # CANT BE MORE THAN 70 - 68 (start and end point)

gdf_routes_dk_crs = gdf_routes_wgs84.to_crs(f.DENMARK_CRS)
# buffer bikelanes
gdf_routes_DK_buffered = f.buffer_gdf(gdf_routes_dk_crs, BUFFER)

print("Number of forest areas already on the path: ", len(forest_areas_already_in_the_road_dk))

# get forest areas along the way
can_be_reached_forest_areas = f.get_only_areas_which_are_crossed_by_bikelane(forest_areas_with_bikelanes_dk, gdf_routes_DK_buffered)
print("Reachabel forest areas: ", len(can_be_reached_forest_areas))
# exttract the already visited forest areas
indices_to_exclude = forest_areas_already_in_the_road_wg84.index
# Drop rows from temp dataframe based on indices_to_exclude
not_visited_forest_areas_dk = can_be_reached_forest_areas.drop(indices_to_exclude)
print("Extra forest areas: ", len(not_visited_forest_areas_dk))
# keep only the X biggest ones
not_visited_forest_areas_dk["area"] = not_visited_forest_areas_dk["geometry"].area
not_visited_forest_areas_dk.sort_values(by="area", ascending=False, inplace=True)
additional_centroids = not_visited_forest_areas_dk.head(ADD_MORE_AREA_TO_BE_VISITED).to_crs(f.CRS).geometry.centroid.to_list()

# add them to the road
# re-calculation

Number of forest areas already on the path:  82
Reachabel forest areas:  931
Extra forest areas:  849


C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_8276\3769387260.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  additional_centroids = not_visited_forest_areas_dk.head(ADD_MORE_AREA_TO_BE_VISITED).to_crs(f.CRS).geometry.centroid.to_list()


In [38]:
indices_to_exclude

Index([  799,   845,  2311,  2547,  3572,  3573,  3690,  3691,  3693,  4044,
        4246,  4284,  4431,  4433,  4511,  4529,  4548,  4556,  4560,  4651,
        4654,  4655,  4730,  5101,  5409,  5537,  5589,  5701,  5720,  5737,
        5747,  5786,  5787,  6374, 10504, 11655, 11943, 12168, 12179, 12190,
       12463, 12517, 12674, 12676, 12678, 12726, 12735, 12888, 12945, 13085,
       13483, 13517, 13578, 13582, 13782, 13875, 13942, 13944, 13945, 13946,
       14028, 14042, 14319, 14320, 14336, 14347, 14350, 14365, 14368, 14588,
       14630, 15158, 15352, 18614, 18836, 18945, 19632, 19666, 20108, 20179,
       20294, 20587],
      dtype='int64')

In [39]:
forest_areas_already_in_the_road_wg84

,geometry
799,"POLYGON ((9.47141 56.05546, 9.47126 56.05543, ..."
845,"POLYGON ((9.46662 56.01053, 9.46675 56.01025, ..."
2311,"POLYGON ((9.52705 56.49185, 9.52713 56.49168, ..."
2547,"POLYGON ((9.60740 56.55431, 9.60808 56.55445, ..."
3572,"POLYGON ((9.45489 56.06006, 9.45445 56.05990, ..."
...,...
19666,"POLYGON ((9.45366 55.17937, 9.45375 55.17939, ..."
20108,"POLYGON ((9.43899 55.70470, 9.43898 55.70465, ..."
20179,"POLYGON ((9.43888 55.69251, 9.43865 55.69260, ..."
20294,"POLYGON ((9.36289 55.82754, 9.36320 55.82752, ..."


In [70]:
additional_centroids = not_visited_forest_areas_dk.head(30).to_crs(f.CRS).geometry.centroid.to_list()

C:\Users\Gergo Gyori\AppData\Local\Temp\ipykernel_26268\1199225711.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  additional_centroids = not_visited_forest_areas_dk.head(30).to_crs(f.CRS).geometry.centroid.to_list()


In [71]:
additional_centroids

[<POINT (10.017 57.133)>,
 <POINT (9.424 56.122)>,
 <POINT (9.515 56.121)>,
 <POINT (9.374 55.945)>,
 <POINT (9.344 55.887)>,
 <POINT (9.321 55.847)>,
 <POINT (9.298 56.319)>,
 <POINT (9.471 56.036)>,
 <POINT (9.444 55.626)>,
 <POINT (9.377 55.916)>,
 <POINT (9.361 56.422)>,
 <POINT (9.52 56.519)>,
 <POINT (9.627 56.568)>,
 <POINT (9.349 56.193)>,
 <POINT (9.336 56.405)>,
 <POINT (9.304 56.238)>,
 <POINT (9.488 55.279)>,
 <POINT (9.556 56.501)>,
 <POINT (9.366 55.9)>,
 <POINT (9.41 55.53)>,
 <POINT (9.407 55.951)>,
 <POINT (9.578 55.422)>,
 <POINT (9.754 56.62)>,
 <POINT (9.289 56.338)>,
 <POINT (9.684 56.588)>,
 <POINT (9.458 56.105)>,
 <POINT (9.465 55.703)>,
 <POINT (9.838 56.827)>,
 <POINT (9.42 55.681)>,
 <POINT (9.422 55.94)>]

In [72]:
imp.reload(f)
additional_centroids_coordinates = f.list_of_points_to_coordinates(additional_centroids)

In [73]:
expanded_coordinates = [coordinates[0]] + additional_centroids_coordinates + [coordinates[1]]

In [74]:
coordinates

[[10.217285, 57.421294], [9.074707, 54.933455]]

In [75]:
print(expanded_coordinates)

[[10.217285, 57.421294], [10.016676071827142, 57.13293867436792], [9.42382807086874, 56.12246062946854], [9.515214859955613, 56.12088120271865], [9.37427767990157, 55.94489449315024], [9.344413341167487, 55.88694119909416], [9.320932708780509, 55.84722806492282], [9.29760340483933, 56.31949525793867], [9.471040313493434, 56.0363688619634], [9.444492031275127, 55.62565274075366], [9.37711845193633, 55.91596152156489], [9.361143458207621, 56.421916838372866], [9.519670203229149, 56.518939911842345], [9.626808823984483, 56.56845847107988], [9.348623342420257, 56.19286988012288], [9.33633596618151, 56.40493901657003], [9.303650044916903, 56.237782320217065], [9.488081829647097, 55.279238392629175], [9.556243239247706, 56.501418157201314], [9.365649472948695, 55.90037892001191], [9.410213057680718, 55.530449975749946], [9.407131053119144, 55.951498897202335], [9.578346971517684, 55.42228136468772], [9.753634814971809, 56.61973176243156], [9.289036059566836, 56.337503225678276], [9.683947620

In [79]:

radiuses = [1000] * len(expanded_coordinates)  # 1000 meters for each point

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=expanded_coordinates,
                           profile='cycling-regular',
                           format='geojson',
                           radiuses=radiuses,
                           optimize_waypoints=True,
                           optimized=False)

gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

[[10.217285, 57.421294], [9.074707, 54.933455]]


In [80]:
# so whe have 
print(coordinates)


radiuses = [1000] * len(expanded_coordinates)  # 1000 meters for each point

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=expanded_coordinates,
                           profile='cycling-regular',
                           format='geojson',
                           radiuses=radiuses,
                           optimize_waypoints=True)

gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

[[10.217285, 57.421294], [9.074707, 54.933455]]


In [33]:
coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point
    [11.61442, 55.60587],    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point,    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

coordinates = [
    [11.581726, 55.606109],  # Original start point
    [11.699829, 55.592143],  # Original end point
    [11.68209, 55.60313],    # Additional point
    [11.61442, 55.60587],    # Additional point
    [11.57688, 55.61297],    # Additional point
    [11.61690, 55.60043],    # Additional point
    [11.62806, 55.60910]     # Additional point
]

print(coordinates)

radiuses = [1000] * len(coordinates)  # 1000 meters for each point

# Make a request to the OpenRouteService API for cycling routes
routes = client.directions(coordinates=coordinates,
                           profile='cycling-regular',
                           format='geojson',
                           radiuses=radiuses,
                           optimize_waypoints=True,
                           optimized=True)

gdf_routes = gpd.GeoDataFrame.from_features(routes['features'])
gdf_routes.crs = 'EPSG:4326'
gdf_routes.explore()

[[11.581726, 55.606109], [11.699829, 55.592143], [11.68209, 55.60313], [11.61442, 55.60587], [11.57688, 55.61297], [11.6169, 55.60043], [11.62806, 55.6091]]


ApiError: 400 ({'error': {'code': 2002, 'message': "Parameter 'optimized' has incorrect value or format."}, 'info': {'engine': {'build_date': '2024-03-21T13:55:54Z', 'version': '8.0.0'}, 'timestamp': 1713521172354}})

In [38]:
# order dataframe by area


,geometry,area
12,"POLYGON ((668339.239 6165079.848, 668301.084 6...",661055.688138
0,"POLYGON ((664431.353 6164998.727, 664388.994 6...",586086.185607
13,"POLYGON ((662791.525 6166271.966, 662891.192 6...",307482.938071
11,"POLYGON ((665174.376 6164834.719, 665156.992 6...",261281.925078
3,"POLYGON ((665822.575 6165773.013, 665818.828 6...",246963.305850
8,"POLYGON ((665408.762 6164249.303, 665424.525 6...",96766.298642
9,"POLYGON ((663327.827 6165918.931, 663330.928 6...",75707.187159
10,"MULTIPOLYGON (((664602.799 6165966.937, 664600...",68029.188812
4,"POLYGON ((668384.427 6164316.293, 668399.865 6...",59271.682936
2,"POLYGON ((671600.937 6163874.231, 671576.807 6...",35898.211838


In [34]:
# check which forest area is along the way
touched_forests.explore()

In [ ]:
# pick the coordinate

In [ ]:
# re-plan